# The problem

We want to be able to take a "JS component" and use it from python, in a jupyter notebook.

By "JS component" I mean "some JS+HTML+CSS that defines an element, view, or full 'app' whose functionality can be reused in various places". 

The idea is to be able to facilitate reuse between front-end dev and back-end dev
(in particular, analysts and ML engineers). 
We'd like, for instance, an analyst working within a notebook to import a python interface 
to some rich data-explorer component written in ReactJS, feed it what it needs (e.g. a data source/target), 
and then use the component's features fully, enabling the work done in it to then be 
used back in the notebook. 

For example, saw we take a [forced-directed graph](https://towardsdatascience.com/large-graph-visualization-tools-and-approaches-2b8758a1cd59) or [tsne viz](https://github.com/karpathy/tsnejs) tool that visualizes data, enabling us to view, filter and select data points. 
We'd like our analysts to be able to provide their data, or point to a source where the data can be found, 
and then once they find an select the data they want to work with, using the features of the tool,
be able to extract that data and get it into some python collection that they can then continue their analysis with. 

Here's a proof-of-concept solution for the example described above, along with other instances of this class.

Use some persisted data store and a platform-independent format as the middleware for py and js communication.
* From python, we write the data to a DB or files in a JS-friendly format (like json or csv).
* JS can then read this data from that source and do it's work
* When we want to communicate some data back to python, we do so again, through the "shared" data store

We could already get far by automating the strategy above as much as possible:
Not obliging the user to set up the data store themselves or have to define the codecs 
necessary to communicate to the data middleware (at least not for the most common data types an analyst might use). 
Perhaps this approach could be further extended to allow some in-memory data store to be used, 
which would improve communication speed. 

We'd like to be able to do this with any JS component, but only through a specific normalized interface. 
That is, the automation of the PY-JS communication setup will be done only for very specific (but open-closed)
python and JS interfaces. To use a third-party JS component, we will need to wrap it into a facade that 
is compliant with our normalized interface that we will rely on, and the python interface will be totally determined by this.
This facade needs to be open-closed though. 
That is, it needs to be able to accomodate any "lines of communication" between python and the JS component. 

Note: Some solutions work in some environments, and not in others. 
Typically, I've seen some things work in my browser-based notebook, but not in VScode notebook. 
We don't want to target covering all environments, but at least browser-based and VScode, which should cover a lot of users. 



## Two-way JS-PY communication

Integrating JavaScript components with a Python notebook and enabling two-way communication can be a bit complex, but it's definitely possible. Here's a high-level overview of how you can achieve this:
1.	**Embedding JavaScript in Jupyter Notebook**: You can use the `IPython` core display module to embed JavaScript code in a cell.
2.	**Creating a JavaScript Component**: Define your JavaScript component and its functionality. This can be a simple button or a more complex interactive component.
3.	**Communication from Python to JavaScript**: You can call a JavaScript function from Python using the `IPython` display module.
4.	**Communication from JavaScript to Python**: This is the tricky part. Jupyter provides a `Jupyter.notebook.kernel.execute` method that allows JavaScript to run Python code.


# Solutions

## Just IPython: Works on browser, not in VScode

In [70]:
from IPython.display import display, HTML, Javascript

js_code = """
function sendToPython(){
    var data = document.querySelector("#myInput").value;
    var kernel = IPython.notebook.kernel;
    kernel.execute("data_from_js = '" + data + "'");
}

document.querySelector("#myButton").addEventListener("click", function(){
    sendToPython();
});
"""

HTML_code = """
<input type="text" id="myInput" placeholder="Enter some text">
<button id="myButton">Submit</button>
<script type="text/Javascript">{}</script>
""".format(js_code)

data_from_js = None   # not even necessary to initialize, but makes checking no throw an error, but print None instead

In [71]:
# Running this cell should display a text box and a button. Enter some text in the text box and click the button. 
# The text should be sent to Python and stored in the variable data_from_js. 
# The next cell will print the value of data_from_js...
# ... if it works (works on browser but not VSCode, for me)
display(HTML(HTML_code))

In [72]:
print(data_from_js)

None


## Embedding Javascript

In [3]:
# alert not showing in VSCode notebook!

from IPython.display import display, HTML, Javascript

# Define a simple JavaScript function
js_code = """
function sayHello() {
    alert('Hello from JavaScript!');
    // Call a Python function from JavaScript
    Jupyter.notebook.kernel.execute('hello_from_python()');
}
"""

# Embed the JavaScript code in the notebook
display(Javascript(js_code))


<IPython.core.display.Javascript object>

In [4]:
# Button showing, but alert not showing in VSCode notebook!

from IPython.display import display, HTML

# Define a simple JavaScript function within an HTML script tag
js_code = """
<script>
function sayHello() {
    alert('Hello from JavaScript!');
    // Call a Python function from JavaScript
    Jupyter.notebook.kernel.execute('hello_from_python()');
}
</script>
<button onclick="sayHello()">Click Me!</button>
"""

# Embed the JavaScript code in the notebook using HTML
display(HTML(js_code))


In [6]:
# Both button and alert showing in VSCode notebook!
# (But not what we're looking for -- we want to be able to have two way communication with a react JS function)
import ipywidgets as widgets
from IPython.display import display

def on_button_click(button):
    print("Button clicked!")

button = widgets.Button(description="Click Me!")
button.on_click(on_button_click)
display(button)

Button(description='Click Me!', style=ButtonStyle())

Button clicked!
Button clicked!


In [9]:
from IPython.display import display, HTML, Javascript

js_code = """
function sendToPython(data){
    var kernel = IPython.notebook.kernel;
    kernel.execute("data_from_js = '" + data + "'");
}

document.querySelector("#myButton").addEventListener("click", function(){
    sendToPython(this.innerHTML);
});
"""

HTML("""
<button id="myButton">Click me!</button>
<script type="text/Javascript">{}</script>
""".format(js_code))

In [8]:
def python_callback(data):
    print(f"Received data from JS: {data}")

# This will be called automatically when the button is clicked
data_from_js = None

if data_from_js:
    python_callback(data_from_js)

In [13]:
import ipywidgets as widgets
from IPython.display import display

def on_button_click(b):
    print(b)
    print(f"{dir(b)}")
    d[key] = text.value

def my_function(d, key):
    text = widgets.Text(description='What you want to tell python:')
    button = widgets.Button(description='Submit')
    button.on_click(on_button_click)

    display(text)
    display(button)

d = dict()

my_function(d, 'key')

Text(value='', description='What you want to tell python:')

Button(description='Submit', style=ButtonStyle())

Button(description='Submit', style=ButtonStyle())
['__annotations__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_active_widgets', '_add_notifiers', '_call_widget_constructed', '_click_handlers', '_comm_changed', '_compare', '_control_comm', '_cross_validation_lock', '_default_keys', '_dom_classes', '_gen_repr_from_keys', '_get_embed_state', '_get_trait_default_generator', '_handle_button_msg', '_handle_control_comm_msg', '_handle_custom_msg', '_handle_msg', '_holding_sync', '_is_numpy', '_lock_property', '_log_default', '_model_id', '_model_module', '_model_module_version', '_model_name', '_msg_callbacks', '_notify_observers', '_notify_trait', '_

NameError: name 'key' is not defined

In [42]:
from IPython.display import display, HTML, Javascript

js_code = """
function sendToPython(){
    var data = document.querySelector("#myInput").value;
    var kernel = IPython.notebook.kernel;
    kernel.execute("data_from_js = '" + data + "'");
}

document.querySelector("#myButton").addEventListener("click", function(){
    sendToPython();
});
"""

HTML_code = """
<input type="text" id="myInput" placeholder="Enter some text">
<button id="myButton">Submit</button>
<script type="text/Javascript">{}</script>
""".format(js_code)

display(HTML(HTML_code))


In [44]:
print(data_from_js)

None


In [14]:
from IPython.display import HTML

def my_function(html, d, key):
    display(HTML(html))

    js_code = """
    const input = document.querySelector('input');
    const button = document.querySelector('button');

    button.addEventListener('click', () => {
        const value = input.value;
        const kernel = IPython.notebook.kernel;
        kernel.execute(`d['${key}'] = '${value}'`);
    });
    """

    display(HTML(f'<script>{js_code}</script>'))

html = """
<div>
  <label for="input">What you want to tell python:</label>
  <input type="text" id="input" name="input">
  <button id="submit">Submit</button>
</div>
"""

d = {}
key = 'my_key'

my_function(html, d, key)

html


None


In [37]:
from IPython.display import HTML

def my_function(html, d, key):
    display(HTML(html))

    js_code = """
    const input = document.querySelector('input');
    const button = document.querySelector('button');

    button.addEventListener('click', () => {
        const value = input.value;
        const kernel = IPython.notebook.kernel;
        kernel.execute(`d['${key}'] = '${value}'`);
    });
    """

    display(HTML(f'<script>{js_code}</script>'))

html = """
<div>
  <label for="input">What you want to tell python:</label>
  <input type="text" id="input" name="input">
  <button id="submit">Submit</button>
</div>
"""

d = {}

from dol import TextFiles

d = TextFiles('/Users/thorwhalen/Dropbox/py/proj/i/jy/misc/data')
key = 'my_key'

my_function(html, d, key)


In [35]:
d[key] = 'foob'

In [36]:
list(d)

['my_key']

In [31]:
pwd

'/Users/thorwhalen/Dropbox/py/proj/i/jy/misc'

In [45]:
from IPython.display import display, Javascript
import ipywidgets as widgets
from IPython.display import display
from traitlets import Unicode

class DataReceiver(widgets.DOMWidget):
    _view_name = Unicode('DataReceiverView').tag(sync=True)
    _view_module = Unicode('dataReceiver').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = Unicode().tag(sync=True)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.value = ""
        self.observe(self.on_value_change, 'value')

    def on_value_change(self, change):
        print(f"Received data from JS: {change['new']}")


In [46]:
%%javascript

require.undef('dataReceiver');

define('dataReceiver', ["@jupyter-widgets/base"], function(widgets) {
    var DataReceiverView = widgets.DOMWidgetView.extend({
        render: function() {
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },

        events: {
            'click #myButton': 'handle_click'
        },

        handle_click: function() {
            var data = document.querySelector("#myInput").value;
            this.model.set('value', data);
            this.touch();
        },

        value_changed: function() {
            var old_value = this.model.previous('value');
            var new_value = this.model.get('value');
        },
    });

    return {
        DataReceiverView: DataReceiverView
    };
});


<IPython.core.display.Javascript object>

In [47]:
receiver = DataReceiver()

HTML_code = """
<input type="text" id="myInput" placeholder="Enter some text">
<button id="myButton">Submit</button>
"""

display(HTML(HTML_code))
display(receiver)

DataReceiver()

In [48]:
from IPython.display import display, HTML, Javascript

js_code = """
function sendToPython(){
    var data = document.querySelector("#myInput").value;
    var kernel = IPython.notebook.kernel;
    kernel.execute("data_from_js = '" + data + "'");
    kernel.execute("print('Received data from JS: ' + data_from_js)");
}

document.querySelector("#myButton").addEventListener("click", function(){
    sendToPython();
});
"""

HTML_code = """
<input type="text" id="myInput" placeholder="Enter some text">
<button id="myButton">Submit</button>
<script type="text/Javascript">{}</script>
""".format(js_code)

display(HTML(HTML_code))


In [49]:
js_code = """
function getInputValueAndSendToPython(){
    var data = document.querySelector("#myInput").value;
    return data;
}
"""

display(Javascript(js_code))


<IPython.core.display.Javascript object>

In [52]:
def handle_input_value(value):
    print(f"Received data from JS: {value}")

HTML_code = """
<input type="text" id="myInput" placeholder="Enter some text">
<button onclick="IPython.notebook.kernel.execute('handle_input_value("' + getInputValueAndSendToPython() + '")')">Submit</button>
"""

display(HTML(HTML_code))

In [53]:
import ipywidgets as widgets

# Create a text input widget
text_input = widgets.Text(
    value='',
    placeholder='Enter some text',
    description='Input:',
    disabled=False
)

# Create a button widget
submit_button = widgets.Button(description="Submit")

# Define the button click callback
def on_button_click(button):
    print(f"Received data: {text_input.value}")

submit_button.on_click(on_button_click)

# Display the widgets
display(text_input, submit_button)

Text(value='', description='Input:', placeholder='Enter some text')

Button(description='Submit', style=ButtonStyle())

Received data: asdf


In [54]:
import ipywidgets as widgets
from traitlets import Unicode, validate

class CustomTextInput(widgets.DOMWidget):
    _view_name = Unicode('CustomTextInputView').tag(sync=True)
    _view_module = Unicode('customTextInput').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    value = Unicode().tag(sync=True)


In [58]:
%%javascript

require.undef('customTextInput');

define('customTextInput', ["@jupyter-widgets/base"], function(widgets) {
    var CustomTextInputView = widgets.DOMWidgetView.extend({
        render: function() {
            this.inputElement = document.createElement('input');
            this.inputElement.type = 'text';
            this.inputElement.placeholder = 'Enter some text';
            
            this.el.appendChild(this.inputElement);
            
            this.inputElement.addEventListener('change', this.valueChanged.bind(this));
            
            this.valueChanged();
        },
        
        valueChanged: function() {
            this.model.set('value', this.inputElement.value);
            this.touch();
        },
    });
    
    return {
        CustomTextInputView: CustomTextInputView
    };
});


<IPython.core.display.Javascript object>

In [61]:
custom_input = CustomTextInput()
display(custom_input)

CustomTextInput()

In [62]:
print(custom_input.value)


In [63]:
import ipywidgets as widgets
from traitlets import Unicode

class CustomTextInput(widgets.DOMWidget):
    _view_name = Unicode('CustomTextInputView').tag(sync=True)
    _view_module = Unicode('customTextInput').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    value = Unicode().tag(sync=True)

In [64]:
%%javascript

require.undef('customTextInput');

define('customTextInput', ["@jupyter-widgets/base"], function(widgets) {
    var CustomTextInputView = widgets.DOMWidgetView.extend({
        render: function() {
            // Create input element
            this.inputElement = document.createElement('input');
            this.inputElement.type = 'text';
            this.inputElement.placeholder = 'Enter some text';
            this.el.appendChild(this.inputElement);
            
            // Create submit button
            this.submitButton = document.createElement('button');
            this.submitButton.innerHTML = 'Submit';
            this.submitButton.onclick = this.handleSubmit.bind(this);
            this.el.appendChild(this.submitButton);
        },
        
        handleSubmit: function() {
            this.model.set('value', this.inputElement.value);
            this.touch();
        },
    });
    
    return {
        CustomTextInputView: CustomTextInputView
    };
});


<IPython.core.display.Javascript object>

In [67]:
custom_input = CustomTextInput()
display(custom_input)

CustomTextInput()

In [66]:
print(custom_input.value)

In [68]:
import ipywidgets as widgets

# Create a text input widget
text_input = widgets.Text(
    value='',
    placeholder='Enter some text',
    description='Input:',
    disabled=False
)

# Create a button widget
submit_button = widgets.Button(description="Submit")

# Define the button click callback
def on_button_click(button):
    global captured_value
    captured_value = text_input.value
    print(f"Received data: {captured_value}")

submit_button.on_click(on_button_click)

# Display the widgets
display(text_input, submit_button)


Text(value='', description='Input:', placeholder='Enter some text')

Button(description='Submit', style=ButtonStyle())

Received data: bloo


# Calling JS mermaid from python, in a notebook

## Demo

In [1]:
import uuid

_mermaid_display_template = """
<script type="module">
    async function initMermaid_{unique_id}() {{
        const module = await import('https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs');
        const mermaid = module.default;
        mermaid.initialize({{ startOnLoad: true }});
        mermaid.init(undefined, document.getElementById('mermaid_{unique_id}'));
    }}
    
    if (document.readyState === 'loading') {{
        document.addEventListener('DOMContentLoaded', initMermaid_{unique_id});
    }} else {{
        initMermaid_{unique_id}();
    }}
</script>

<pre class="mermaid" id="mermaid_{unique_id}">
    {code}
</pre>
"""


def mermaid(code: str, *, prefix='graph', suffix='', egress=None):
    if egress is None:
        egress = lambda x: x
    code_ = f"{prefix}\n{code}\n{suffix}"
    unique_id = str(uuid.uuid4()).replace("-", "")
    html = _mermaid_display_template.format(code=code_, unique_id=unique_id)
    return egress(html)


In [2]:
code = "A --> B --> C"

In [5]:
html = mermaid(code)
assert isinstance(html, str)
# print(html)

You can define an `egress` to tell the function to return the result of `egress(html)` instead. This is useful, for example, to save the html, or wrap it in an object that can render it. For example, `IPython.display.HTML` will wrap it in a displayable `HTML` instance.

In [6]:
from IPython.display import HTML

mermaid(code, egress=HTML)

`mermaid` is setup to be a framework through `functools.partial`. 
For example, to get a function that does left-to-right graphs (the default is top-to-bottom),
and returning an `HTML` instance systematically, you can do the following:

In [8]:
from functools import partial
from IPython.display import display, HTML

lr_graph = partial(mermaid, prefix='graph LR', egress=HTML)

In [9]:
lr_graph(code)